In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/MyDrive/colab_projects/nlp/imdb/nbs/

/content/gdrive/MyDrive/colab_projects/nlp/imdb/nbs


In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [4]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [5]:
df = pd.read_csv("../data/imdb.csv")

In [6]:
df_train = df[:25000]
df_test = df[25000:]

In [7]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 128

In [8]:
labels_index = {'positive':1, 'negative':0}

In [9]:
df_train.review.values

array(["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to

In [10]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df_train.review.values)
train_sequences = tokenizer.texts_to_sequences(df_train.review.values)
test_sequences = tokenizer.texts_to_sequences(df_test.review.values)
word_index = tokenizer.word_index

In [11]:
input_sequences = []
for line in df_train.review.values:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [12]:
input_sequences[10]

[27, 4, 1, 77, 2019, 45, 1006, 12, 100, 147, 39, 318]

In [13]:
dict_decode = {}
for key, value in tokenizer.word_index.items():
    dict_decode[value] = key

In [14]:
input_sequences[1:3]

[[27, 4, 1], [27, 4, 1, 77]]

In [15]:
input_sequences

[[27, 4],
 [27, 4, 1],
 [27, 4, 1, 77],
 [27, 4, 1, 77, 2019],
 [27, 4, 1, 77, 2019, 45],
 [27, 4, 1, 77, 2019, 45, 1006],
 [27, 4, 1, 77, 2019, 45, 1006, 12],
 [27, 4, 1, 77, 2019, 45, 1006, 12, 100],
 [27, 4, 1, 77, 2019, 45, 1006, 12, 100, 147],
 [27, 4, 1, 77, 2019, 45, 1006, 12, 100, 147, 39],
 [27, 4, 1, 77, 2019, 45, 1006, 12, 100, 147, 39, 318],
 [27, 4, 1, 77, 2019, 45, 1006, 12, 100, 147, 39, 318, 3335],
 [27, 4, 1, 77, 2019, 45, 1006, 12, 100, 147, 39, 318, 3335, 425],
 [27, 4, 1, 77, 2019, 45, 1006, 12, 100, 147, 39, 318, 3335, 425, 450],
 [27, 4, 1, 77, 2019, 45, 1006, 12, 100, 147, 39, 318, 3335, 425, 450, 26],
 [27,
  4,
  1,
  77,
  2019,
  45,
  1006,
  12,
  100,
  147,
  39,
  318,
  3335,
  425,
  450,
  26,
  3076],
 [27,
  4,
  1,
  77,
  2019,
  45,
  1006,
  12,
  100,
  147,
  39,
  318,
  3335,
  425,
  450,
  26,
  3076,
  33],
 [27,
  4,
  1,
  77,
  2019,
  45,
  1006,
  12,
  100,
  147,
  39,
  318,
  3335,
  425,
  450,
  26,
  3076,
  33,
  23],
 [27,
 

In [16]:
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1

In [17]:
num_words

20001

In [18]:
class KerasBatchGenerator(object):
    def __init__(self, input_sequences, batch_size):
        self.input_sequences = input_sequences
        self.batch_size = batch_size
        self.current_idx = 0
    
    def generate(self):
        while True:
            if (i + 1)*self.batch_size < len(self.input_sequences):
                in_seq = input_sequences[i*self.batch_size: (i + 1)*self.batch_size]
                in_seq = np.array(pad_sequences(in_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='pre'))
                
            else:
                in_seq = input_sequences[i*self.batch_size:]
                in_seq = np.array(pad_sequences(in_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='pre'))
                
            X = in_seq[:,:-1]
            y = in_seq[:,-1]
            y = to_categorical(y, num_classes=num_words)
            self.current_idx += len(self.input_sequences) // self.batch_size
            yield (X, y)

In [19]:
gen = KerasBatchGenerator(input_sequences, 2048)

In [20]:
next(gen.generate())[1].shape

(2048, 20001)

In [21]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model, Sequential

In [22]:
input_len = MAX_SEQUENCE_LENGTH - 1
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(MAX_NUM_WORDS + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [23]:
model.fit(gen.generate(), steps_per_epoch=len(input_sequences)//2048, epochs=1, verbose=1)

2777/2777 [==============================] - 2268s 816ms/step - loss: 3.8148


In [24]:
data = next(gen.generate())
prediction = model.predict(data[0])

In [25]:
prediction.shape

(2048, 20001)

In [26]:
predictions = np.argmax(prediction, axis=1)

In [27]:
print(data[0][0,:])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [28]:
print(predictions[0])

26


In [29]:
MAX_SEQUENCE_LENGTH

1000

In [30]:
print([dict_decode[x] for x in data[0][7,:] if x > 0]), print(dict_decode[predictions[7]])

['when', 'oh', 'when', 'will', 'write', 'a', 'decent', 'movie', 'based', 'around', 'computers', 'i', 'cannot', 'believe', 'people', 'actually', 'consider', 'this', 'movie', 'to', 'be', 'a', 'credible', 'story', 'br', 'br', 'no', 'computer', 'operating', 'system', 'could', 'ever', 'survive', 'wit', 'that', 'sort', 'of', 'annoying', 'it', 'may', 'look', 'good', 'on', 'a', 'movie', 'screen', 'but', 'if', 'you', 'actually', 'tried', 'using', 'it', 'for', 'any', 'length', 'of', 'time', 'you', 'would', 'go', 'nuts', 'br', 'br', 'as', 'for', 'tracing', 'people', 'the', 'way', 'she', 'did', 'it', 'simply', 'cannot', 'be', 'done', 'that', 'way', 'network', 'security', 'alone', 'would', 'prevent', 'that', 'from', 'happening', 'the', 'key', 'stroke', 'was', 'laughable', 'to', 'say', 'the', 'least', 'br', 'br', 'regarding', 'the', 'software', 'that', 'was', 'supposedly', 'being', 'no', 'system', 'administrator', 'would', 'allow', 'such', 'a', 'critical', 'piece', 'of', 'software', 'to', 'be', 'on'

(None, None)

In [31]:
new_model = Sequential()
for layer in model.layers[:-2]:
    new_model.add(layer)
new_model.add(LSTM(128, dropout=0.2))
new_model.add(Dense(2, activation='sigmoid'))
print(new_model.summary())
new_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 128)         2560128   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 2,691,970
Trainable params: 2,691,970
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
X_train = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
y_train = to_categorical(np.asarray(df_train.sentiment.map(labels_index)))
y_test = to_categorical(np.asarray(df_test.sentiment.map(labels_index)))

In [33]:
new_model.fit(X_train, y_train, batch_size=128, epochs=5)

score, acc = new_model.evaluate(X_test, y_test)
print('Test accuracy:', acc)

Epoch 1/5
196/196 [==============================] - 22s 105ms/step - loss: 0.6124 - accuracy: 0.6411
Epoch 2/5
196/196 [==============================] - 21s 105ms/step - loss: 0.4067 - accuracy: 0.8216
Epoch 3/5
196/196 [==============================] - 21s 105ms/step - loss: 0.2396 - accuracy: 0.9067
Epoch 4/5
196/196 [==============================] - 21s 105ms/step - loss: 0.1835 - accuracy: 0.9335
Epoch 5/5
782/782 [==============================] - 16s 20ms/step - loss: 0.3588 - accuracy: 0.8748
Test accuracy: 0.8748000264167786
